# Import packages

In [1]:
import sys
import plotly
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from matplotlib import pyplot as plt


In [ ]:
print("hello world")
print(sys.version_info)

hello world
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)


# COVID-19 Project – The Socially Distanced



Research questions:

1) What conditions predict a second wave of COVID-19 infections in a population center?

- Suggestion: What characteristics predict spikes in a second wave of COVID-19 infection in a population center? 

2) Which populations centers can expect a second wave imminently? 

Response variable:

Break into two groups of population centers: 2nd wave and NO 2nd wave. The basis for the categorization should be on the 14-day moving average of new cases per day

Predictive variables:

Disease:

    1) Recovered cases per population

    2) Deaths per population AND/OR deaths per positive cases

    3) Prior hospitalization rate

    4) Prior ICU rate

Testing and Tracing:

    1) Tests conducted per population

    2) Is there a way to quantify tracing protocols or workforce?

Interventions:

    1) Severity level of intervention (create as ordinal factor or dummy variable for different measures in place)

    2) Duration of intervention actions

    3) School closure?

Population Demographics:

    1) Population size AND/OR Population density

    2) Median age

    3) Median income

    4) Racial composition

    5) Transportation statistic (ie airport size)



# machine learning moduling
using data 1


PART - COVID data 1

load COVID data

In [ ]:

df_data = pd.read_csv('https://raw.githubusercontent.com/wlbit666/datathon/master/features_and_labels.csv')

df_X = df_data.drop(['FIPS', 'State', 'Area_Name','Combined_Key', 'case.rate','X8.9.20','labels'], axis=1)
df_X = df_X.fillna(np.nan)

df_X.head()

df_play = df_X[['Female_age85plusr','Total.households','Total.households..Family.households..families.']]
df_play.head()
df_play.dtypes

Female_age85plusr                                   int64
Total.households                                  float64
Total.households..Family.households..families.    float64
dtype: object

In [ ]:
# check all the data type of each column 
for t in df_X.columns: 
  print(df_X[t].dtype, 'is the data type of' + t)


In [ ]:
# locate 'inf' in which row
df_X1 = df_X.round(4)
df_X1.head()
df_X1.index[np.isinf(df_X1).any(1)]

Int64Index([72, 90, 93, 94, 251], dtype='int64')

In [ ]:
# locate 'inf' in which column 
df_X1.columns.to_series()[np.isinf(df_X1).any()]

pop.density    pop.density
dtype: object

In [ ]:
# remove 'inf' from the dataset
df_X = df_X.replace([np.inf, -np.inf], np.nan)
np.any(np.isinf(df_X))


False

clean data 

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy= 'median')
imputer.fit(df_X)
df_pimpute = imputer.transform(df_X)
df_X = pd.DataFrame(df_pimpute, columns=df_X.columns)
df_X.head()

,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018.x,N_POP_CHG_2018,Births_2018,Deaths_2018,NATURAL_INC_2018,INTERNATIONAL_MIG_2018,DOMESTIC_MIG_2018,NET_MIG_2018,RESIDUAL_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less.than.a.high.school.diploma.2014.18,High.school.diploma.only.2014.18,Some.college.or.associate.s.degree.2014.18,Bachelor.s.degree.or.higher.2014.18,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,PCTPOVALL_2018,CI90LBALLP_2018,CI90UBALLP_2018,POV017_2018,CI90LB017_2018,CI90UB017_2018,PCTPOV017_2018,CI90LB017P_2018,CI90UB017P_2018,POV517_2018,...,Total.physician.assistants..2019.,Total.Hospitals..2019.,Internal.Medicine.Primary.Care..2019.,Family.Medicine.General.Practice.Primary.Care..2019.,Pediatrics.Primary.Care..2019.,Obstetrics...Gynecology.Primary.Care..2019.,Geriatrics.Primary.Care..2019.,Total.Primary.Care.Physicians..2019.,Psychiatry.specialists..2019.,Surgery.specialists..2019.,Anesthesiology.specialists..2019.,Emergency.Medicine.specialists..2019.,Radiology.specialists..2019.,Cardiology.specialists..2019.,Oncology..Cancer..specialists..2019.,Endocrinology.Diabetes.and.Metabolism.specialists..2019.,All.Other.Specialties.specialists..2019.,Total.Specialist.Physicians..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,COUNTY.POPULATION.AGENCIES.REPORT.ARRESTS,COUNTY.POPULATION.AGENCIES.REPORT.CRIMES,NUMBER.OF.AGENCIES.IN.COUNTY.REPORT.ARRESTS,NUMBER.OF.AGENCIES.IN.COUNTY.REPORT.CRIMES,COVERAGE.INDICATOR,Total.number.of.UCR..Uniform.Crime.Report..Index.crimes.excluding.arson.,Total.number.of.UCR..Uniform.Crime.Report..index.crimes.reported.including.arson,MURDER,RAPE,ROBBERY,Number.of.AGGRAVATED.ASSAULTS,BURGLRY,LARCENY,MOTOR.VEHICLE.THEFTS,ARSON,POP_ESTIMATE_2018.y,Area,pop.density,pop.density.percentile
0,2.0,2.0,0.0,55601.0,158.0,655.0,532.0,123.0,-2.0,37.0,35.0,0.0,455.0,11.8,9.6,2.2,0.0,0.7,0.6,4204.0,12119.0,10552.0,10291.0,11.3,32.6,28.4,27.7,7587.0,6334.0,8840.0,13.8,11.5,16.1,2509.0,1965.0,3053.0,19.3,15.1,23.5,1891.0,...,6.085786,1.148905,25.992561,21.249061,11.716559,7.632826,0.102378,66.693385,5.755902,8.576976,6.847931,5.289515,7.666952,4.663873,2.741447,0.807646,29.791911,72.142154,6.0,0.0,2.516019e+09,57161.0,54604.0,3.0,3.0,100.000,139.0,1765.0,3.0,15.0,34.0,87.0,447.0,1233.0,85.0,108.0,55601.0,604.49,91.980016,0.709236
1,3.0,2.0,5.0,218022.0,5403.0,2254.0,2228.0,26.0,118.0,5232.0,5350.0,27.0,2190.0,10.5,10.3,0.1,0.5,24.3,24.8,14310.0,40579.0,46025.0,46075.0,9.7,27.6,31.3,31.3,21069.0,17390.0,24748.0,9.8,8.1,11.5,6442.0,4723.0,8161.0,13.9,10.2,17.6,4534.0,...,23.863512,4.505074,101.921730,83.321572,45.942837,29.929751,0.401442,261.517333,22.569976,33.631941,26.852027,20.741184,30.063565,18.287925,10.749732,3.166933,116.819699,282.882982,51.0,368329051.0,2.280863e+08,187467.0,187467.0,14.0,14.0,100.000,446.0,4988.0,8.0,30.0,76.0,332.0,967.0,3829.0,192.0,31.0,218022.0,2027.08,107.554709,0.743949
2,6.0,6.0,3.0,24881.0,-277.0,261.0,324.0,-63.0,12.0,-227.0,-215.0,1.0,2820.0,10.4,12.9,-2.5,0.5,-9.1,-8.6,4901.0,6486.0,4566.0,2220.0,27.0,35.7,25.1,12.2,6788.0,5662.0,7914.0,30.9,25.8,36.0,2242.0,1788.0,2696.0,43.9,35.0,52.8,1411.0,...,2.723340,0.514126,11.631462,9.508784,5.243066,3.415628,0.045813,29.844753,2.575720,3.838128,3.064394,2.367015,3.430899,2.087046,1.226776,0.361415,13.331641,32.283033,5.0,0.0,1.772788e+09,27228.0,25566.0,6.0,4.0,970.113,48.0,474.0,0.0,4.0,8.0,36.0,90.0,362.0,21.0,0.0,24881.0,904.59,27.505279,0.375796
3,1.0,1.0,0.0,22400.0,-155.0,250.0,256.0,-6.0,10.0,-158.0,-148.0,-1.0,2151.0,11.1,11.4,-0.3,0.4,-7.0,-6.6,2650.0,7471.0,384

In [ ]:
g = df_X.columns.to_series().groupby(df_X.dtypes).groups  # group the data type of each column
g

{dtype('float64'): Index(['Rural.urban_Continuum.Code_2013', 'Urban_Influence_Code_2013',
        'Economic_typology_2015', 'POP_ESTIMATE_2018.x', 'N_POP_CHG_2018',
        'Births_2018', 'Deaths_2018', 'NATURAL_INC_2018',
        'INTERNATIONAL_MIG_2018', 'DOMESTIC_MIG_2018',
        ...
        'Number.of.AGGRAVATED.ASSAULTS', 'BURGLRY', 'LARCENY',
        'MOTOR.VEHICLE.THEFTS', 'ARSON', 'X8.9.20', 'POP_ESTIMATE_2018.y',
        'Area', 'pop.density', 'pop.density.percentile'],
       dtype='object', length=349)}

In [ ]:
df_y = df_data[['labels']]
df_y.head()

,labels
0,1
1,1
2,1
3,1
4,0


In [ ]:
# add more attributes "the proportion of race"

total_pop = df_X['POP_ESTIMATE_2018.y']

race_names = df_X.columns[173: 245]
race_names

for name in race_names: 
  prop_name = str(name) + '_prop'
  df_X[prop_name] = df_X[name] / total_pop

df_X.head()

,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018.x,N_POP_CHG_2018,Births_2018,Deaths_2018,NATURAL_INC_2018,INTERNATIONAL_MIG_2018,DOMESTIC_MIG_2018,NET_MIG_2018,RESIDUAL_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less.than.a.high.school.diploma.2014.18,High.school.diploma.only.2014.18,Some.college.or.associate.s.degree.2014.18,Bachelor.s.degree.or.higher.2014.18,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,PCTPOVALL_2018,CI90LBALLP_2018,CI90UBALLP_2018,POV017_2018,CI90LB017_2018,CI90UB017_2018,PCTPOV017_2018,CI90LB017P_2018,CI90UB017P_2018,POV517_2018,...,NHAA_MALE_prop,NHAA_FEMALE_prop,NHNA_MALE_prop,NHNA_FEMALE_prop,NHTOM_MALE_prop,NHTOM_FEMALE_prop,NHWAC_MALE_prop,NHWAC_FEMALE_prop,NHBAC_MALE_prop,NHBAC_FEMALE_prop,NHIAC_MALE_prop,NHIAC_FEMALE_prop,NHAAC_MALE_prop,NHAAC_FEMALE_prop,NHNAC_MALE_prop,NHNAC_FEMALE_prop,H_MALE_prop,H_FEMALE_prop,HWA_MALE_prop,HWA_FEMALE_prop,HBA_MALE_prop,HBA_FEMALE_prop,HIA_MALE_prop,HIA_FEMALE_prop,HAA_MALE_prop,HAA_FEMALE_prop,HNA_MALE_prop,HNA_FEMALE_prop,HTOM_MALE_prop,HTOM_FEMALE_prop,HWAC_MALE_prop,HWAC_FEMALE_prop,HBAC_MALE_prop,HBAC_FEMALE_prop,HIAC_MALE_prop,HIAC_FEMALE_prop,HAAC_MALE_prop,HAAC_FEMALE_prop,HNAC_MALE_prop,HNAC_FEMALE_prop
0,2.0,2.0,0.0,55601.0,158.0,655.0,532.0,123.0,-2.0,37.0,35.0,0.0,455.0,11.8,9.6,2.2,0.0,0.7,0.6,4204.0,12119.0,10552.0,10291.0,11.3,32.6,28.4,27.7,7587.0,6334.0,8840.0,13.8,11.5,16.1,2509.0,1965.0,3053.0,19.3,15.1,23.5,1891.0,...,0.005342,0.006709,0.000360,0.000414,0.008669,0.007967,0.370425,0.387871,0.095934,0.106455,0.004334,0.004694,0.007806,0.008849,0.000647,0.000845,0.015467,0.014190,0.012626,0.011547,0.001421,0.001457,0.000288,0.000144,0.000072,0.000126,0.000234,0.000108,0.000827,0.000809,0.013273,0.012230,0.001942,0.001834,0.000647,0.000594,0.000342,0.000306,0.000396,0.000216
1,3.0,2.0,5.0,218022.0,5403.0,2254.0,2228.0,26.0,118.0,5232.0,5350.0,27.0,2190.0,10.5,10.3,0.1,0.5,24.3,24.8,14310.0,40579.0,46025.0,46075.0,9.7,27.6,31.3,31.3,21069.0,17390.0,24748.0,9.8,8.1,11.5,6442.0,4723.0,8161.0,13.9,10.2,17.6,4534.0,...,0.004183,0.007160,0.000225,0.000284,0.007733,0.008293,0.409706,0.436594,0.045537,0.049000,0.006921,0.006949,0.005986,0.009050,0.000550,0.000660,0.024319,0.022149,0.021750,0.019989,0.000766,0.000798,0.000642,0.000381,0.000096,0.000064,0.000110,0.000050,0.000954,0.000867,0.022585,0.020714,0.001188,0.001160,0.001188,0.000881,0.000316,0.000261,0.000257,0.000156
2,6.0,6.0,3.0,24881.0,-277.0,261.0,324.0,-63.0,12.0,-227.0,-215.0,1.0,2820.0,10.4,12.9,-2.5,0.5,-9.1,-8.6,4901.0,6486.0,4566.0,2220.0,27.0,35.7,25.1,12.2,6788.0,5662.0,7914.0,30.9,25.8,36.0,2242.0,1788.0,2696.0,43.9,35.0,52.8,1411.0,...,0.002090,0.002452,0.000764,0.000322,0.006270,0.005185,0.243680,0.221776,0.258832,0.229774,0.004542,0.003296,0.002894,0.003135,0.000924,0.000442,0.023472,0.019292,0.018488,0.015795,0.001768,0.001889,0.002050,0.001125,0.000000,0.000000,0.000442,0.000322,0.000723,0.000161,0.018970,0.015916,0.002331,0.001969,0.002492,0.001246,0.000040,0.000000,0.000442,0.000322
3,1.0,1.0,0.0,22400.0,-155.0,250.0,256.0,-6.0,10.0,-158.0,-148.0,-1.0,2151.0,11.1,11.4,-0.3,0.4,-7.0,-6.6,2650.0,7471.0,3846.0,1813.0,16.8,47.3,24.4,11.5,4400.0,3445.0,5355.0,21.8,17.1,26.5,1238.0,923.0,1553.0,27.8,20.7,34.9,840.0,...,0.000804,0.001518,0.000223,0.000045,0.004866,0.005357,0.382991,0.372188,0.133348,0.083571,0.003973,0.004063,0.001652,0.002098,0.000536,0.000268,0.014955,0.011295,0.012545,0.009911,0.001161,0.000580,0.000134,0.000402,0.000045,0.000000,0.000759,0.000134,0.000313,0.000268,0.012768,0.010179,0.001384,0.000714,0.000313,0.000625,0.000045,0.000000,0.000804,0.000134
4,1.0,1.0,0.0,57840.0,13.0,68

In [ ]:
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

para_grid = [
             {'n_estimators':[10, 50, 100, 200], 'max_features': [100, 150, 200, 400]}, 
             {'bootstrap': [False], 'n_estimators':[10, 50, 100, 200], 'max_features': [100, 150, 200, 400]}
]

forest_c = RandomForestClassifier(oob_score = True)

grid_search = GridSearchCV(forest_c, para_grid, cv=5)

grid_search.fit(df_X, df_y)

grid_search.best_params_

{'max_features': 100, 'n_estimators': 50}

In [ ]:
grid_search.cv_results_

{'mean_fit_time': array([1.59116373e+00, 7.64358444e+00, 1.53624762e+01, 3.05340981e+01,
        2.36136675e+00, 1.16138870e+01, 2.34081949e+01, 4.68409731e+01,
        3.09416590e+00, 1.59847747e+01, 3.14417590e+01, 6.31116029e+01,
        6.32464986e+00, 3.29235762e+01, 6.54197897e+01, 1.31238272e+02,
        8.93926620e-03, 8.65421295e-03, 8.41636658e-03, 8.65087509e-03,
        8.69870186e-03, 8.83879662e-03, 8.87689590e-03, 8.66489410e-03,
        8.75763893e-03, 8.76207352e-03, 8.78930092e-03, 8.76235962e-03,
        8.77699852e-03, 8.94308090e-03, 9.82136726e-03, 8.90784264e-03]),
 'mean_score_time': array([0.00408149, 0.00965648, 0.01776257, 0.03116307, 0.00408401,
        0.00987005, 0.01707559, 0.03156891, 0.00455365, 0.00985456,
        0.01681886, 0.03088522, 0.00406022, 0.00990353, 0.01783543,
        0.03189898, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.

In [ ]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = df_X.columns
sorted(zip(feature_importance, attributes), reverse=True)

[(0.06808525115990353, 'Feb.Temp.Max...F'),
 (0.06029522604103519, 'Dec.Temp.Max...F'),
 (0.05621308710603921, 'May.Temp.Min...F'),
 (0.03397582053699841, 'Feb.Temp.Min...F'),
 (0.03131467472352066, 'Feb.Temp.AVG...F'),
 (0.01976441233992768, 'Oct.Temp.Max...F'),
 (0.019745155591565106, 'May.Temp.AVG...F'),
 (0.018586433215036783, 'NH_MALE_prop'),
 (0.013799475053282568, 'NHWAC_MALE_prop'),
 (0.013588008185256184, 'Oct.Temp.AVG...F'),
 (0.013244452043262743,
  'Percent.of.adults.with.less.than.a.high.school.diploma.2014.18'),
 (0.011597191809888822, 'NHWAC_FEMALE_prop'),
 (0.008882088634076355, 'NHIAC_FEMALE_prop'),
 (0.008593872712379283, 'HBA_FEMALE_prop'),
 (0.007865100909939776, 'Mar.Temp.Min...F'),
 (0.007592495520700235, 'May.Temp.Max...F'),
 (0.007486466049612411, 'R_birth_2018'),
 (0.007452920547143487, 'CI90LBALLP_2018'),
 (0.007359868603207253, 'NATURAL_INC_2018'),
 (0.007150087127241491, 'NHWA_FEMALE_prop'),
 (0.007081852542035944,
  'Fraction.of.Active.Physicians.Who.Are.Ag

# machine learning moduling
using data 2

In [2]:
df_data = pd.read_csv('https://raw.githubusercontent.com/wlbit666/datathon/master/RawData.csv')

df_X = df_data.drop(['FIPS', 'State', 'County_Equivalent','POP_ESTIMATE_2018.x','pop.density.percentile',
                     'case.rate','case.density.percentile','response'], axis=1)
df_X = df_X.fillna(np.nan)

df_X.head()


,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,71.2,2,2,3,7.1,47.4,25.9,19.6,4.5,57599.0,94.6,32.9,49.1,58.6,64.6,73.0,68.8,273.3,88.2,1.360002,3.868825,10.565360,0.606088,0.962983,10.922250,0.576523,0.420249,6,0.0,9998928686,0.501775,0.193226,0.594986,0.211787,0.022110,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.971900,0.009909,0.004844,0.019042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15.4,7,11,0,7.9,31.7,41.6,18.8,2.4,49034.0,82.1,35.7,52.3,55.7,70.0,77.5,74.4,214.5,80.3,0.344697,0.889539,5.142650,0.138991,0.255743,2.462912,0.111192,0.517045,0,695652693.0,9994548428,0.497576,0.218907,0.573093,0.207999,0.035521,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960283,0.013612,0.004755,0.051277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37.1,7,9,0,22.9,41.3,23.7,12.1,5.5,31831.0,58.5,47.7,60.4,71.0,76.3,79.4,79.0,236.3,79.0,0.961480,1.925617,5.609521,0.297475,0.446212,4.982700,0.217794,0.324035,7,0.0,9987095326,0.472468,0.257067,0.572026,0.170907,0.023230,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.184366,0.0,0.0,0.184366,0.0,0.708210,0.260816,0.004609,0.027040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.9,9,10,1,22.9,22.3,33.8,21.0,2.0,65771.0,113.1,39.2,55.1,59.6,71.9,79.1,79.4,206.5,80.5,0.167842,0.530464,2.971426,0.060092,0.089101,1.365530,0.060092,0.288025,0,0.0,9985022466,0.503398,0.289906,0.559257,0.150837,0.021880,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.812365,2.486325,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960550,0.017073,0.004475,0.163932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,529.1,7,8,2,19.2,24.4,39.4,16.9,3.9,34442.0,47.4,42.6,55.0,65.7,66.8,71.5,70.5,231.8,81.7,0.136029,0.539459,1.404085,0.063356,0.091307,1.311380,0.054505,0.044722,11,0.0,9957679861,0.468498,0.199849,0.613659,0.186492,0.023690,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.898185,0.057208,0.018901,0.048387,NaN,NaN

In [3]:
df_X = df_X.drop(df_X.columns.to_series()["Unnamed: 68":"Unnamed: 83"], axis=1)
df_X.head()


,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic
0,71.2,2,2,3,7.1,47.4,25.9,19.6,4.5,57599.0,94.6,32.9,49.1,58.6,64.6,73.0,68.8,273.3,88.2,1.360002,3.868825,10.565360,0.606088,0.962983,10.922250,0.576523,0.420249,6,0.0,9998928686,0.501775,0.193226,0.594986,0.211787,0.022110,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.971900,0.009909,0.004844,0.019042
1,15.4,7,11,0,7.9,31.7,41.6,18.8,2.4,49034.0,82.1,35.7,52.3,55.7,70.0,77.5,74.4,214.5,80.3,0.344697,0.889539,5.142650,0.138991,0.255743,2.462912,0.111192,0.517045,0,695652693.0,9994548428,0.497576,0.218907,0.573093,0.207999,0.035521,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960283,0.013612,0.004755,0.051277
2,37.1,7,9,0,22.9,41.3,23.7,12.1,5.5,31831.0,58.5,47.7,60.4,71.0,76.3,79.4,79.0,236.3,79.0,0.961480,1.925617,5.609521,0.297475,0.446212,4.982700,0.217794,0.324035,7,0.0,9987095326,0.472468,0.257067,0.572026,0.170907,0.023230,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.184366,0.0,0.0,0.184366,0.0,0.708210,0.260816,0.004609,0.027040
3,6.9,9,10,1,22.9,22.3,33.8,21.0,2.0,65771.0,113.1,39.2,55.1,59.6,71.9,79.1,79.4,206.5,80.5,0.167842,0.530464,2.971426,0.060092,0.089101,1.365530,0.060092,0.288025,0,0.0,9985022466,0.503398,0.289906,0.559257,0.150837,0.021880,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.812365,2.486325,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960550,0.017073,0.004475,0.163932
4,529.1,7,8,2,19.2,24.4,39.4,16.9,3.9,34442.0,47.4,42.6,55.0,65.7,66.8,71.5,70.5,231.8,81.7,0.136029,0.539459,1.404085,0.063356,0.091307,1.311380,0.054505,0.044722,11,0.0,9957679861,0.468498,0.199849,0.613659,0.186492,0.023690,0.13264,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.898185,0.057208,0.018901,0.048387


In [4]:
np.any(np.isnan(df_X))


True

In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy= 'median')
imputer.fit(df_X)
df_pimpute = imputer.transform(df_X)
df_X = pd.DataFrame(df_pimpute, columns=df_X.columns)
df_X.head()

,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic
0,71.2,2.0,2.0,3.0,7.1,47.4,25.9,19.6,4.5,57599.0,94.6,32.9,49.1,58.6,64.6,73.0,68.8,273.3,88.2,1.360002,3.868825,10.565360,0.606088,0.962983,10.922250,0.576523,0.420249,6.0,0.0,9.998929e+09,0.501775,0.193226,0.594986,0.211787,0.022110,0.13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.971900,0.009909,0.004844,0.019042
1,15.4,7.0,11.0,0.0,7.9,31.7,41.6,18.8,2.4,49034.0,82.1,35.7,52.3,55.7,70.0,77.5,74.4,214.5,80.3,0.344697,0.889539,5.142650,0.138991,0.255743,2.462912,0.111192,0.517045,0.0,695652693.0,9.994548e+09,0.497576,0.218907,0.573093,0.207999,0.035521,0.13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960283,0.013612,0.004755,0.051277
2,37.1,7.0,9.0,0.0,22.9,41.3,23.7,12.1,5.5,31831.0,58.5,47.7,60.4,71.0,76.3,79.4,79.0,236.3,79.0,0.961480,1.925617,5.609521,0.297475,0.446212,4.982700,0.217794,0.324035,7.0,0.0,9.987095e+09,0.472468,0.257067,0.572026,0.170907,0.023230,0.13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.184366,0.0,0.0,0.184366,0.0,0.708210,0.260816,0.004609,0.027040
3,6.9,9.0,10.0,1.0,22.9,22.3,33.8,21.0,2.0,65771.0,113.1,39.2,55.1,59.6,71.9,79.1,79.4,206.5,80.5,0.167842,0.530464,2.971426,0.060092,0.089101,1.365530,0.060092,0.288025,0.0,0.0,9.985022e+09,0.503398,0.289906,0.559257,0.150837,0.021880,0.13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.812365,2.486325,0.0,0.000000,0.0,0.0,0.000000,0.0,0.960550,0.017073,0.004475,0.163932
4,529.1,7.0,8.0,2.0,19.2,24.4,39.4,16.9,3.9,34442.0,47.4,42.6,55.0,65.7,66.8,71.5,70.5,231.8,81.7,0.136029,0.539459,1.404085,0.063356,0.091307,1.311380,0.054505,0.044722,11.0,0.0,9.957680e+09,0.468498,0.199849,0.613659,0.186492,0.023690,0.13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.898185,0.057208,0.018901,0.048387


In [6]:
df_y = df_data[['response']]
df_y.head()

,response
0,0
1,0
2,0
3,0
4,0


split the dataset 80: 20 

In [24]:
from sklearn.model_selection import train_test_split

df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

In [25]:
df_X_train.head()

,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic
1292,108.2,5.0,8.0,5.0,8.2,30.1,32.2,29.5,2.5,73981.0,91.9,80.0,80.0,80.0,80.0,80.0,80.0,273.7,104.7,2.132774,13.964590,27.218256,4.062426,1.371069,34.936866,1.015607,1.117167,9.0,0.000000e+00,3.021061e+09,0.493630,0.218333,0.580109,0.201558,0.028184,0.096544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027727,0.027727,0.027727,0.0,0.0,0.013863,0.027727,0.027727,0.013863,0.027727,0.330237,0.006613,0.310385,0.112071
2302,199.0,2.0,2.0,4.0,7.0,30.6,31.1,31.4,3.0,65984.0,118.7,29.9,44.2,52.3,62.7,72.0,67.9,305.7,119.7,8.936974,44.684871,138.831271,6.009345,5.238916,80.895025,3.389887,5.238916,14.0,1.388221e+09,1.434080e+09,0.487836,0.187001,0.608212,0.204787,0.025559,0.153480,75.0,75.0,69.0,0.0,0.0,68.0,72.0,129.0,60.0,137.0,0.009698,0.009698,1.895951,0.0,0.0,0.111527,0.009698,0.358824,0.358824,0.111527,0.955326,0.009868,0.013626,0.017384
761,756.6,2.0,2.0,0.0,9.7,29.3,29.6,31.4,4.0,63297.0,93.6,31.8,47.2,56.4,64.7,72.5,68.7,304.1,94.8,8.426427,22.075649,37.418106,5.190997,7.734824,88.262849,4.078073,1.319610,36.0,6.350573e+09,4.500010e+09,0.487325,0.214574,0.614265,0.171162,0.028143,0.134850,140.0,140.0,0.0,0.0,150.0,146.0,0.0,144.0,67.0,140.0,0.205986,0.205986,0.000000,0.0,0.0,0.025748,0.000000,0.090119,0.205986,0.205986,0.780837,0.123624,0.049154,0.073692
2194,113.6,4.0,5.0,3.0,14.8,51.8,20.9,12.5,4.7,47648.0,78.3,36.3,51.9,62.4,67.0,74.9,71.4,273.3,88.2,2.324262,6.611877,18.056340,1.035813,1.645751,18.666280,0.985285,0.718212,10.0,3.391687e+08,1.565986e+09,0.489724,0.221886,0.561032,0.217083,0.029856,0.132640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.021635,0.000000,0.043269,0.000000,0.971009,0.009368,0.006166,0.016572
1241,13.1,9.0,10.0,5.0,17.1,44.6,26.1,12.2,5.2,34367.0,63.2,43.9,59.0,66.0,72.9,76.8,76.7,236.3,79.0,0.532797,1.067065,3.108472,0.164843,0.247265,2.761124,0.120689,0.179561,0.0,0.000000e+00,3.138075e+09,0.501608,0.187756,0.523899,0.288344,0.030942,0.132640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.964622,0.002107,0.001885,0.017301


In [26]:
df_y_train.head()

,response
1292,0
2302,0
761,0
2194,0
1241,0


In [28]:
np.shape(df_X_train)

(2512, 60)

start the modeling

In [29]:
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

para_grid = [
             {'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}, 
             {'bootstrap': [False], 'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}
]

forest_c = RandomForestClassifier(oob_score = True)

grid_search = GridSearchCV(forest_c, para_grid, cv=5)

grid_search.fit(df_X_train, df_y_train)

grid_search.best_params_

{'max_features': 10, 'n_estimators': 200}

In [30]:
grid_search.cv_results_

{'mean_fit_time': array([1.15025806e-01, 5.35878325e-01, 1.08602424e+00, 2.15231705e+00,
        1.93886042e-01, 9.78122187e-01, 1.91558523e+00, 3.89464092e+00,
        3.74054289e-01, 1.87430191e+00, 3.69473829e+00, 7.49524202e+00,
        5.79044199e-01, 2.82452912e+00, 5.67273440e+00, 1.12971667e+01,
        3.25956345e-03, 3.10759544e-03, 3.21922302e-03, 3.11117172e-03,
        3.12151909e-03, 3.02066803e-03, 3.08537483e-03, 3.10201645e-03,
        3.00722122e-03, 3.06243896e-03, 3.07397842e-03, 3.13010216e-03,
        3.07970047e-03, 3.35345268e-03, 3.06129456e-03, 3.04036140e-03]),
 'mean_score_time': array([0.00320492, 0.0088203 , 0.01618156, 0.02885199, 0.0031291 ,
        0.00831933, 0.01481519, 0.02763057, 0.00307984, 0.00817471,
        0.01518092, 0.02807093, 0.00319066, 0.00819616, 0.01444116,
        0.02766399, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.

In [31]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = df_X_train.columns
sorted(zip(feature_importance, attributes), reverse=True)

[(0.091529966381399, 'May.Temp.AVG...F'),
 (0.06646548331432656, 'percent.black'),
 (0.06418025961064587, 'Mar.Temp.AVG...F'),
 (0.048305486736226096,
  'Percent.of.adults.with.less.than.a.high.school.diploma.2014.18'),
 (0.04640559006283374, 'Jun.Temp.AVG...F'),
 (0.04381880538267546, 'percent.hispanic'),
 (0.04049685870058582, 'Apr.Temp.AVG...F'),
 (0.03400205316770723, 'Aug.Temp.AVG...F'),
 (0.032770455989119644, 'Jul.Temp.AVG...F'),
 (0.02712412905176007, 'percent.white'),
 (0.024971370422252726, 'percent.age.65.plus'),
 (0.02430421799764121, 'percent.age.0.17'),
 (0.020308528060670777, 'cases_shelter_in_place'),
 (0.019419762160287644,
  'Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18'),
 (0.01669486676068617, 'percent.age.18.64'),
 (0.01668500141056042, 'percent.age.85.plus'),
 (0.016579943158917504, 'Median_Household_Income_2018'),
 (0.015177679887915609, 'Density.per.square.mile.of.land.area...Population'),
 (0.014502491201710443, 'percent.asian'),
 (0.

try dataset 2 with normalization 

In [7]:
from sklearn import preprocessing
c_names = df_X.columns
df_X_scale = preprocessing.scale(df_X)
df_X_scale = pd.DataFrame(df_X_scale, columns=c_names)
df_X_scale.head()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning:

Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.



,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic
0,-0.189645,-1.110479,-0.933534,0.655117,-0.994773,1.826074,-0.929358,-0.210159,0.253902,0.345454,0.255866,-0.770615,-0.649496,-0.477690,-0.873095,-0.542392,-0.817641,1.494147,0.773596,-0.206229,-0.255473,-0.319082,-0.198192,-0.195148,-0.274722,-0.194984,-0.287642,-0.231646,-0.443517,2.825885,0.027731,-0.792715,0.215957,0.405191,-0.194284,-0.000029,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,-0.19595,-0.251516,-0.075211,-0.101166,-0.105292,-0.302848,-0.129876,-0.188312,-0.355195,-0.254497,0.777508,-0.577138,-0.367392,-0.560255
1,-0.262264,0.736007,1.639534,-0.994055,-0.868476,-0.359717,2.090104,-0.295002,-1.173192,-0.272049,-0.367501,-0.524576,-0.282622,-0.776485,-0.101472,0.246041,-0.018274,-0.370009,0.020897,-0.288008,-0.334629,-0.395730,-0.272855,-0.263606,-0.383520,-0.279988,-0.265094,-0.312150,0.066507,2.824084,-0.156184,-0.049059,-0.350741,0.324544,1.289087,-0.000029,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,-0.19595,-0.251516,-0.075211,-0.101166,-0.105292,-0.302848,-0.129876,-0.188312,-0.355195,-0.254497,0.706383,-0.551558,-0.370400,-0.326649
2,-0.234023,0.736007,1.067741,-0.994055,1.499601,0.976817,-1.352468,-1.005564,0.933471,-1.512319,-1.544417,0.529874,0.646029,0.799918,0.798754,0.578935,0.638348,0.321124,-0.102964,-0.238328,-0.307101,-0.389131,-0.247522,-0.245169,-0.351112,-0.260515,-0.310055,-0.218228,-0.443517,2.821020,-1.255967,1.055960,-0.378383,-0.465139,-0.070442,-0.000029,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,-0.19595,-0.251516,-0.075211,-0.101166,-0.105292,0.428182,-0.129876,-0.188312,0.278999,-0.254497,-0.837029,1.156180,-0.375332,-0.502290
3,-0.273326,1.474601,1.353637,-0.444331,1.499601,-1.668407,0.589989,-0.061684,-1.445019,0.934624,1.178448,-0.217028,0.038393,-0.374657,0.170025,0.526373,0.695446,-0.623636,0.039953,-0.302253,-0.344169,-0.426419,-0.285466,-0.279736,-0.397633,-0.289323,-0.318443,-0.312150,-0.443517,2.820168,0.098832,2.006871,-0.708886,-0.892424,-0.219815,-0.000029,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,

In [8]:
from sklearn.model_selection import train_test_split

df_X_scale_train, df_X_scale_test, df_y_train, df_y_test = train_test_split(df_X_scale, df_y, test_size=0.2, random_state=42)
df_X_scale_train.head()

,Density.per.square.mile.of.land.area...Population,Rural.urban_Continuum.Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,Percent.of.adults.with.less.than.a.high.school.diploma.2014.18,Percent.of.adults.with.a.high.school.diploma.only.2014.18,Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18,Percent.of.adults.with.a.bachelor.s.degree.or.higher.2014.18,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Mar.Temp.AVG...F,Apr.Temp.AVG...F,May.Temp.AVG...F,Jun.Temp.AVG...F,Jul.Temp.AVG...F,Aug.Temp.AVG...F,Total.Active.Patient.Care.Physicians.per.100000.Population.2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100000.Population.2018..AAMC.,Critical.Care.Medicine..AAMC.,Emergency.Medicine..AAMC.,Family.Medicine.General.Practice..AAMC.,Geriatric.Medicine.....AAMC.,Infectious.Disease..AAMC.,Internal.Medicine..AAMC.,Pulmonary.Disease..AAMC.,Total.Hospitals..2019.,ICU.Beds,transit_scores...population.weighted.averages.aggregated.from.town.city.level.to.county,crime_rate_per_100000,percent.male,percent.age.0.17,percent.age.18.64,percent.age.65.plus,percent.age.85.plus,percent.disabled,days_closing_of_public_venues,days_gathering_size_10_0,days_gathering_size_100_26,days_gathering_size_25_11,days_gathering_size_500_101,days_non-essential_services_closure,days_religious_gatherings_banned,days_school_closure,days_shelter_in_place,days_social_distancing,cases_closing_of_public_venues,cases_gathering_size_10_0,cases_gathering_size_100_26,cases_gathering_size_25_11,cases_gathering_size_500_101,cases_non-essential_services_closure,cases_religious_gatherings_banned,cases_school_closure,cases_shelter_in_place,cases_social_distancing,percent.white,percent.black,percent.asian,percent.hispanic
1292,-0.141493,-0.002587,0.781845,1.754565,-0.821114,-0.582472,0.282273,0.839776,-1.105235,1.526533,0.121218,3.368102,2.893134,1.727213,1.327459,0.684060,0.781092,1.506829,2.345686,-0.143986,0.012760,-0.083700,0.354284,-0.155647,0.034136,-0.114775,-0.125297,-0.191394,-0.443517,-0.042798,-0.329035,-0.065696,-0.169144,0.187416,0.477521,-2.132148,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,0.018551,-0.092755,-0.066027,-0.101166,-0.105292,-0.247878,-0.060080,0.317624,-0.307507,-0.096949,-3.151312,-0.599909,9.979714,0.113912
2302,-0.023326,-1.110479,-0.933534,1.204841,-1.010561,-0.512861,0.070718,1.041278,-0.765451,0.949981,1.457717,-1.034227,-1.211272,-1.126797,-1.144592,-0.717599,-0.946111,2.521336,3.774860,0.404063,0.828963,1.493908,0.665487,0.218743,0.625216,0.318945,0.834853,-0.124307,0.574270,-0.695224,-0.582802,-0.972990,0.558310,0.256154,0.187157,1.230937,1.471668,1.207084,1.316095,-0.218331,-0.314676,1.437034,1.894356,1.931686,1.191717,2.112161,-0.120924,-0.195986,0.552837,-0.101166,-0.105292,0.139367,-0.105464,6.359276,0.879114,0.379220,0.676032,-0.577423,-0.069991,-0.572271
761,0.702336,-1.110479,-0.933534,-0.994055,-0.584307,-0.693850,-0.217765,1.041278,-0.085882,0.756258,0.205996,-0.867273,-0.867328,-0.704362,-0.858806,-0.629995,-0.831916,2.470610,1.402432,0.362940,0.228262,0.060471,0.534679,0.460336,0.719975,0.444659,-0.078138,0.170875,4.212466,0.565215,-0.605182,-0.174554,0.714974,-0.459714,0.472970,0.130531,3.138551,2.650859,-0.360293,-0.218331,3.530300,3.587132,-0.372686,2.213331,1.379690,2.168564,1.397626,0.927956,-0.075211,-0.101166,-0.105292,-0.200753,-0.129876,1.456121,0.353371,0.915962,-0.392346,0.208429,1.133155,-0.164215
2194,-0.134466,-0.371885,-0.075844,0.655117,0.220840,2.438652,-1.890971,-0.963143,0.389816,-0.371974,-0.557005,-0.471854,-0.328481,-0.086165,-0.530152,-0.209498,-0.446507,1.494147,0.773596,-0.128562,-0.182593,-0.213200,-0.129503,-0.129059,-0.175124,-0.120314,-0.218233,-0.177976,-0.194853,-0.640996,-0.500140,0.037187,-0.662963,0.517929,0.662448,-0.000029,-0.451659,-0.458811,-0.360293,-0.218331,-0.314676,-0.437411,-0.372686,-0.490456,-0.419474,-0.463606,-0.195950,-0.251516,-0.075211,-0

In [36]:
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

para_grid = [
             {'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}, 
             {'bootstrap': [False], 'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}
]

forest_c = RandomForestClassifier(oob_score = True)

grid_search = GridSearchCV(forest_c, para_grid, cv=5)

grid_search.fit(df_X_scale_train, df_y_train)

grid_search.best_params_

{'max_features': 10, 'n_estimators': 100}

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([1.12895393e-01, 5.35028458e-01, 1.07197886e+00, 2.14034252e+00,
        1.95934105e-01, 9.82766008e-01, 1.94009795e+00, 3.89300961e+00,
        3.75370550e-01, 1.87593622e+00, 3.72652106e+00, 7.51528697e+00,
        5.67024708e-01, 2.84236474e+00, 5.75043416e+00, 1.15617084e+01,
        4.12940979e-03, 3.39217186e-03, 3.25317383e-03, 3.81894112e-03,
        3.89308929e-03, 3.50270271e-03, 3.87983322e-03, 4.03466225e-03,
        3.91893387e-03, 3.55854034e-03, 3.25417519e-03, 3.38234901e-03,
        3.30343246e-03, 3.39937210e-03, 3.30424309e-03, 3.39765549e-03]),
 'mean_score_time': array([0.00362844, 0.00858655, 0.01592546, 0.02893238, 0.00310545,
        0.00862646, 0.01494637, 0.02800465, 0.00314555, 0.00840569,
        0.01471324, 0.02843194, 0.003161  , 0.00841818, 0.01508684,
        0.02872925, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.

In [38]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = df_X_train.columns
sorted(zip(feature_importance, attributes), reverse=True)

[(0.07933804575378066, 'May.Temp.AVG...F'),
 (0.07719330379390105, 'Mar.Temp.AVG...F'),
 (0.06114944253607111, 'percent.black'),
 (0.05077020148884296,
  'Percent.of.adults.with.less.than.a.high.school.diploma.2014.18'),
 (0.04696767081761256, 'percent.hispanic'),
 (0.040829241539476625, 'Apr.Temp.AVG...F'),
 (0.04046765564235245, 'Jun.Temp.AVG...F'),
 (0.03428778530721252, 'Aug.Temp.AVG...F'),
 (0.030777021136723273, 'Jul.Temp.AVG...F'),
 (0.02908546372557221, 'percent.white'),
 (0.02510984691973523, 'percent.age.65.plus'),
 (0.02327957331166302, 'percent.age.0.17'),
 (0.021051520451883382,
  'Percent.of.adults.completing.some.college.or.associate.s.degree.2014.18'),
 (0.020385321419688688, 'cases_shelter_in_place'),
 (0.01820101990699951, 'Median_Household_Income_2018'),
 (0.016514146255185137, 'Density.per.square.mile.of.land.area...Population'),
 (0.016073542021142247, 'percent.age.18.64'),
 (0.015702295909616536, 'Infectious.Disease..AAMC.'),
 (0.01539020563247105, 'percent.asian'

try normalized with ANN

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings("ignore")

para_grid = [
             {'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}, 
             {'bootstrap': [False], 'n_estimators':[10, 50, 100, 200], 'max_features': [10, 20, 40, 60]}
]

para_grid2 = {
    'hidden_layer_sizes': [(50,50,50), (50,100,), (100,)],
    'activation': ['tanh', 'relu'],
    'alpha': [0.01, 0.05],
    'learning_rate': ['constant','adaptive'],
}

forest_c = RandomForestClassifier(oob_score = True)
ann_c = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
ann_c2 = MLPClassifier(solver='lbfgs',random_state=1)


grid_search = GridSearchCV(ann_c2, para_grid2, cv=5)

grid_search.fit(df_X_scale_train, df_y_train)

grid_search.best_params_

{'activation': 'relu',
 'alpha': 0.05,
 'hidden_layer_sizes': (50, 100),
 'learning_rate': 'constant'}

In [10]:
grid_search.cv_results_

{'mean_fit_time': array([1.9457593 , 1.94775782, 1.91197634, 1.91210418, 1.3169467 ,
        1.31068125, 5.14392323, 5.15398798, 4.73793235, 4.69547915,
        2.19755454, 2.18500323, 1.46095395, 1.45814705, 1.46863575,
        1.47720933, 1.29477277, 1.29783912, 3.09183784, 3.08876977,
        3.18874216, 3.1923048 , 2.09265771, 2.10023074]),
 'mean_score_time': array([0.00652542, 0.0063695 , 0.00752411, 0.00634289, 0.00503068,
        0.00491142, 0.0061132 , 0.00611725, 0.00601993, 0.00751767,
        0.0049418 , 0.0049542 , 0.00343747, 0.00339079, 0.0054553 ,
        0.00344892, 0.00320659, 0.00296192, 0.00349431, 0.00349402,
        0.00352559, 0.00352368, 0.00298724, 0.00304723]),
 'mean_test_score': array([0.84036577, 0.84036577, 0.83876423, 0.83876423, 0.85031405,
        0.85031405, 0.83359366, 0.83359366, 0.839565  , 0.839565  ,
        0.84871488, 0.84871488, 0.84910537, 0.84910537, 0.85388387,
        0.85388387, 0.84831172, 0.84831172, 0.84433004, 0.84433004,
        0.853

In [11]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = df_X_train.columns
sorted(zip(feature_importance, attributes), reverse=True)

AttributeError: ignored